In [15]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
pip install -U scikit-learn

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/79/3d/02d5d3ed359498fec3abdf65407d3c07e3b8765af17464969055aaec5171/scikit_learn-1.4.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/packages/d4/a1/d4adf25b6d2bef8d0ad1682829dcfcba97f3f96bb5b6f137bc3e41003cc7/scipy-1.13.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.6 kB ? 


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

In [18]:
# Read the CSV file
df = pd.read_csv('datasets/diseases_.csv')

# Create a dictionary to map diseases to numbers
disease_mapping = {disease: idx for idx, disease in enumerate(df['Disease'].unique())}
df['Disease'] = df['Disease'].map(disease_mapping)

# Extract all unique symptom words
unique_symptom_words = set()
for i in range(1, 18):
    symptom_col = f'Symptom_{i}'
    if symptom_col in df.columns:
        df[symptom_col] = df[symptom_col].fillna('')
        for symptoms in df[symptom_col]:
            unique_symptom_words.update(symptoms.split('_'))

unique_symptom_words = sorted(unique_symptom_words)  # Sort for consistency

# Initialize a new DataFrame with the disease column
df_encoded = pd.DataFrame(df['Disease'])

# Create binary columns for each unique symptom word
for word in unique_symptom_words:
    df_encoded[word] = 0

# Fill the binary columns based on the presence of symptom words
for i, row in df.iterrows():
    for j in range(1, 18):
        symptom_col = f'Symptom_{j}'
        if symptom_col in df.columns:
            symptoms = row[symptom_col].split('_')
            for symptom in symptoms:
                if symptom in df_encoded.columns:
                    df_encoded.at[i, symptom] = 1

# Save the encoded DataFrame to a new CSV file
df_encoded.to_csv('datasets/df_encoded.csv', index=False)

C:\Users\ashvi\AppData\Local\Temp\ipykernel_15600\4233943276.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_encoded[word] = 0
C:\Users\ashvi\AppData\Local\Temp\ipykernel_15600\4233943276.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_encoded[word] = 0
C:\Users\ashvi\AppData\Local\Temp\ipykernel_15600\4233943276.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.con

In [19]:
print("Disease Mapping:", disease_mapping)

Disease Mapping: {'Fungal infection': 0, 'Allergy': 1, 'GERD': 2, 'Chronic cholestasis': 3, 'Drug Reaction': 4, 'Peptic ulcer diseae': 5, 'AIDS': 6, 'Diabetes ': 7, 'Gastroenteritis': 8, 'Bronchial Asthma': 9, 'Hypertension ': 10, 'Migraine': 11, 'Cervical spondylosis': 12, 'Paralysis (brain hemorrhage)': 13, 'Jaundice': 14, 'Malaria': 15, 'Chicken pox': 16, 'Dengue': 17, 'Typhoid': 18, 'hepatitis A': 19, 'Hepatitis B': 20, 'Hepatitis C': 21, 'Hepatitis D': 22, 'Hepatitis E': 23, 'Alcoholic hepatitis': 24, 'Tuberculosis': 25, 'Common Cold': 26, 'Pneumonia': 27, 'Dimorphic hemmorhoids(piles)': 28, 'Heart attack': 29, 'Varicose veins': 30, 'Hypothyroidism': 31, 'Hyperthyroidism': 32, 'Hypoglycemia': 33, 'Osteoarthristis': 34, 'Arthritis': 35, '(vertigo) Paroymsal  Positional Vertigo': 36, 'Acne': 37, 'Urinary tract infection': 38, 'Psoriasis': 39, 'Impetigo': 40}


In [20]:
df_encoded

,Disease,,abdominal,abnormal,acidity,acute,altered,anxiety,back,belly,...,tongue,transfusion,unsterile,urine,veins,vessels,vision,walking,wasting,weakness
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,36,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4916,37,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4917,38,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4918,39,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
import numpy as np
import re

# Ensure symptom_df has all symptom columns
all_symptoms = df_encoded.drop('Disease', axis=1).columns.tolist()

# Function to preprocess the input text
def preprocess_input_text(input_text):
    input_symptoms = set(re.findall(r'\b\w+\b', input_text.lower()))
    symptom_vector = np.zeros(len(all_symptoms))
    for i, symptom in enumerate(all_symptoms):
        if symptom in input_symptoms:
            symptom_vector[i] = 1
    return symptom_vector


In [22]:
# Create a correct disease mapping dictionary
disease_mapping = {disease: idx for idx, disease in enumerate(df_encoded['Disease'].unique())}
reverse_disease_mapping = {idx: disease for disease, idx in disease_mapping.items()}


In [23]:
# Split the data into training and testing sets
X = df_encoded.drop('Disease', axis=1)
y = df_encoded['Disease']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(disease_mapping), activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_test, y_test))

# Save the model
model.save('model/disease_prediction_model.h5')


C:\Users\ashvi\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.2946 - loss: 2.8624 - val_accuracy: 1.0000 - val_loss: 0.1808
Epoch 2/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8606 - loss: 0.6050 - val_accuracy: 1.0000 - val_loss: 0.0165
Epoch 3/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9280 - loss: 0.2898 - val_accuracy: 1.0000 - val_loss: 0.0049
Epoch 4/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9531 - loss: 0.1851 - val_accuracy: 1.0000 - val_loss: 0.0018
Epoch 5/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9566 - loss: 0.1511 - val_accuracy: 1.0000 - val_loss: 7.6805e-04
Epoch 6/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9594 - loss: 0.1369 - val_accuracy: 1.0000 - val_loss: 5.2350e-04
Epoch 7/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9732 - loss: 0.1023 - val_accuracy: 1.0000 - val_loss: 2.9008e-04
Epoch 8/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9668 - loss: 0.108

In [25]:
# Load the trained model
model = tf.keras.models.load_model('model/disease_prediction_model.h5')

# Function to predict the disease from input text
def predict_disease(input_text):
    input_vector = preprocess_input_text(input_text)
    input_vector = input_vector.reshape(1, -1)
    predictions = model.predict(input_vector)
    disease_idx = np.argmax(predictions)
    confidence = np.max(predictions)
    predicted_disease = reverse_disease_mapping[disease_idx]
    return predicted_disease, confidence

In [29]:
input_text = input("Give symptoms of the patient: ")
predicted_disease, confidence = predict_disease(input_text)
print(f'Predicted Disease: {predicted_disease}, Confidence: {confidence:.2f}\n')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Predicted Disease: 13, Confidence: 1.00

